# One-class SVM の詳細調査

scikit-learn の <b>One-class SVM</b> について、さらに詳細調査を行いました。

結論としては、一定の効果が得られることを確認しました（ただしデフォルトから所定のチューニングが必要）。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定したものを使用します。
'''
csv_file_name = 'test_benefitone_conversation.csv'
copied_csv_file_path = TestTool.copy_testdata_csv(learning_dir, csv_file_name)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]


## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_path = copied_csv_file_path
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
'''
from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, csv_file_path, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/18 PM 02:28:42 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017/03/18 PM 02:28:42 TextArray#__init__ start
2017/03/18 PM 02:28:43 TextArray#to_vec start
2017/03/18 PM 02:28:43 TextArray#to_vec end


### (2-2) 外れデータのTF-IDFベクター

In [5]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
from learning.core.training_set.text_array import TextArray

test_X = [
    '人生相談をしたいのですが？', # featureが１件だけ得られるような質問文（ただしmajority）
    '何か習い事をしたほうがいいですか？', # featureが３件得られるような質問文
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件得られるような質問文
    '有給休暇を取って世界旅行に行きたいと思っています。', # minority featureと判定されそうな質問文
    'これは自然言語の機械学習ですか？', # まったくfeatureが得られないような質問文
    '難解なプログラミング技術は必須？',
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/18 PM 02:28:43 TextArray#__init__ start
2017/03/18 PM 02:28:43 TextArray#to_vec start
2017/03/18 PM 02:28:43 TextArray#to_vec end


### (2-3) 外れデータのfeatureを確認

In [6]:
'''
    feature をダンプするためのツール
'''
def get_item_from_vocabulary(vocabulary, index):
    for k, v in vocabulary.items():
        if v == index:
            return k

    return None

def dump_features(arr, vocabulary):
    features_str = ''

    for i, v in enumerate(arr):
        if v == 0.0:
            continue

        if features_str != '':
            features_str += ' '
        
        item = get_item_from_vocabulary(vocabulary, i)
        features_str += '%s=%0.3f' % (item, v)

    return '[' + features_str + ']'

In [7]:
vocabulary = text_array._vectorizer.vocabulary_

for i, label in enumerate(X_error):
    arr = X_error[i].toarray()[0]
    dump_str = dump_features(arr, vocabulary)
    print('index=%d%s' % (i, dump_str))

index=0[する=1.000]
index=1[いい=0.577 する=0.577 何=0.577]
index=2[いい=0.500 する=0.500 会社=0.500 誰=0.500]
index=3[取る=0.577 思う=0.577 行く=0.577]
index=4[]
index=5[]


In [8]:
for i, label in enumerate(X):
    arr = X[i].toarray()[0]
    cnt = 0
    for v in arr:
        if v == 0.0:
            continue
        cnt = cnt + 1
    print(i, cnt)

0 4
1 5
2 4
3 3
4 3
5 5
6 5
7 7
8 3
9 3
10 3
11 3
12 2
13 3
14 2
15 2
16 3
17 4
18 3
19 3
20 5
21 6
22 3
23 3
24 3
25 4
26 1
27 3
28 3
29 4
30 4
31 3
32 3
33 4
34 4
35 4
36 2
37 2
38 3
39 3
40 4
41 5
42 5
43 5
44 2
45 3
46 3
47 3
48 3
49 3
50 2
51 2
52 3
53 3
54 3
55 2
56 2
57 3
58 3
59 2
60 2
61 3
62 2
63 2
64 2
65 2
66 2
67 3
68 3
69 4
70 4
71 2
72 2
73 2
74 2
75 3
76 2
77 2
78 2
79 3
80 3
81 2
82 2
83 2
84 6
85 3
86 3
87 3
88 3
89 3
90 4
91 6
92 5
93 7
94 5
95 7
96 3
97 3
98 4
99 5
100 3
101 3
102 4
103 5
104 3
105 2
106 2
107 4
108 5
109 5
110 5
111 5
112 2
113 2
114 2
115 2
116 3
117 5
118 3
119 3
120 5
121 3
122 2
123 3
124 3
125 2
126 2
127 2
128 3
129 3
130 3
131 3
132 2
133 2
134 2
135 2
136 2
137 2
138 3
139 2
140 3
141 2
142 2
143 2
144 4
145 4
146 5
147 6
148 2
149 2
150 2
151 3
152 3
153 2
154 2
155 2
156 2
157 2
158 2
159 2
160 2
161 2
162 2
163 2
164 2
165 3
166 3
167 3
168 5
169 3
170 2
171 2
172 2
173 2
174 3
175 3
176 4
177 4
178 4
179 4
180 3
181 4
182 5
183 4
184 4


## (3) One-class SVMで学習（複数の手順を使用して比較）

### (3-1) 非線形カーネルを使用（RBF）

In [9]:
'''
    One-class SVM with non-linear kernel, anomary is half
        fn(i,j) = exp(-gamma*|i-j|^2)
'''
from sklearn import svm
clf_rbf = svm.OneClassSVM(
    kernel='rbf', # RBFカーネルを使用
    gamma=0.1,    # 係数（デフォルト＝[1/feature数]）
    nu=0.5        # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_rbf.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-2) 多項式カーネルを使用

In [10]:
'''
    One-class SVM with Polynomial kernel, anomary is half
        fn(i,j) = (gamma*i'*j + coef0)^degree
'''
from sklearn import svm
clf_poly = svm.OneClassSVM(
    kernel='poly', # Polynomialカーネルを使用
    gamma=0.1,     # 係数 （デフォルト＝[1/feature数]）
    coef0=1.0,     # オフセット（デフォルト＝0）
    degree=2,      # 乗数（デフォルト＝3）
    nu=0.5         # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_poly.fit(X)

OneClassSVM(cache_size=200, coef0=1.0, degree=2, gamma=0.1, kernel='poly',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-3) シグモイドカーネルを使用

In [11]:
'''
    One-class SVM with Sigmoid kernel, anomary is half
        fn(i,j) = tanh(gamma*i'*j + coef0)
'''
from sklearn import svm
clf_sigmoid = svm.OneClassSVM(
    kernel='sigmoid', # シグモイドカーネルを使用
    gamma=0.1,        # 係数（デフォルト＝[1/feature数]）
    coef0=0.5,        # オフセット（デフォルト＝0）
    nu=0.5            # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_sigmoid.fit(X)

OneClassSVM(cache_size=200, coef0=0.5, degree=3, gamma=0.1, kernel='sigmoid',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-4) 線形カーネルを使用

In [12]:
'''
    One-class SVM with linear kernel, anomary is half
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_linear = svm.OneClassSVM(
    kernel='linear', # linearカーネルを使用
    nu=0.5           # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_linear.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

## (4) 外れデータを使って予測

カーネルを変更すると、予測結果が若干変わってきます。

In [13]:
'''
    外れデータを使用して、外れ判定を実行してみます
'''
y_pred_by_rbf = clf_rbf.predict(X_error)
y_pred_by_poly = clf_poly.predict(X_error)
y_pred_by_sigmoid = clf_sigmoid.predict(X_error)
y_pred_by_linear = clf_linear.predict(X_error)

#### 非線形カーネルの場合

 明らかに異常な質問文（まったくfeatureが得られないような質問文）に対して、正常と判定されてしまいます。
 
 これはプロダクションへの選択肢としては無しかと存じます。

In [14]:
y_pred_by_rbf

array([ 1.,  1.,  1., -1.,  1.,  1.])

#### 多項式カーネル、シグモイドカーネル、線形カーネルの場合

 明らかに異常な質問文は、アノマリーと判定されました。

In [15]:
y_pred_by_poly

array([ 1.,  1.,  1., -1., -1., -1.])

In [16]:
y_pred_by_sigmoid

array([ 1.,  1.,  1., -1., -1., -1.])

In [17]:
y_pred_by_linear

array([ 1.,  1.,  1., -1., -1., -1.])

## (5) 結論

### 非線形カーネル以外のモデルを使用すれば、明らかにおかしい質問文に対して、アノマリー検出できるようです。

ただし、前述のレポートのとおり、微妙な質問文に対して効果が弱い感じは変わりません。

質問文のfeatureが、majority featureに含まれると認識された場合などは、意味的におかしな質問文でも、アノマリー検出されないケースがあるかと存じます。

（たとえば [する=1.000] のケースなど）

前処理のIDF条件をきつくするなど、極々普遍的なfeatureを制限すれば、回避できる問題かもしれません。